In [9]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv

In [10]:
weibo_m = pd.read_csv('./data/Weibo_M.csv')
weibo = pd.read_csv('./data/Weibo.csv')
wechat = pd.read_csv('./data/Wechat.csv')
news_forum = pd.read_csv('./data/News_forum.csv')

In [11]:
weibo_m = weibo_m[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
weibo = weibo[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
wechat = wechat[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
news_forum = news_forum[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')

In [12]:
weibo_m.columns = ['CONTENT','AREAS AFFECTED']
weibo.columns = ['CONTENT','AREAS AFFECTED']

In [13]:
data = pd.concat([weibo_m,weibo,wechat,news_forum])

In [14]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('life', 'Life')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('sponsored', 'Sponsored')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('news', 'News')
data = data.reset_index(drop=True)

In [15]:
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')

In [16]:
#data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('News', 'Brands')
# for i in range(len(data)):
#     if ((data['AREAS AFFECTED'][i] != 'Products/Service') and 
#         (data['AREAS AFFECTED'][i] != 'Corporate Brands')):
#         data['AREAS AFFECTED'][i] = 'others'

In [17]:
mypath = './data/processed/'
for the_file in os.listdir(mypath):
    file_path = os.path.join(mypath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [18]:
for x in data['AREAS AFFECTED'].unique():
    np.savetxt(mypath+ x.replace('/',' ') +'.utf8',\
                   data[data['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')

In [19]:
data['AREAS AFFECTED'].value_counts()

Products/Service         9062
Corporate News           3431
Corporate Brands         3306
Life Comprehend          2167
Charity Events           1870
Customer Service         1752
General Mentioned        1558
Sponsored Events         1494
Stocks&Earnings          1293
Ads/Marketing            1220
Health Information        590
Products                  519
Fund                      479
General Info              421
Survey/Questions          370
Employment                364
Recruitment               358
Irrelevant Ads            267
Employee Information      128
Volunteering Activity      96
Contact Information        66
Agent Issues               41
Website Issues             39
Agent                       3
Name: AREAS AFFECTED, dtype: int64